In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# data importing in pcd variable
# ply format
# PLY is a computer file format known as the Polygon File Format 
# or the Stanford Triangle Format
# C:\Users\HDSL36\Desktop\Work\PCL\office_canvas.ply
# C:\Users\HDSL36\Desktop\Work\PCL\AA02.pts
# C:\Users\HDSL36\Desktop\Work\PCL\office_pix4D_raw_data.ply
pcd = o3d.io.read_point_cloud(r"D:\ZED\fused_point_cloud_still.ply")
pcd.paint_uniform_color([0.6, 0.6, 0.6])

[Open3D WARNING] Read PLY failed: unable to read file: D:\ZED\fused_point_cloud_still.ply


PointCloud with 815877 points.

In [3]:
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
downpcd_o = downpcd
pcd = downpcd

In [4]:
segment_models={}
segments={}
max_plane_idx=10
rest=pcd
d_threshold=0.1
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.1,ransac_n=3,num_iterations=5000)
    segments[i]=rest.select_by_index(inliers)
    # Clustering the plane Using DBSCAN
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    
    #  no of cluster found in that plane segment named as candidates
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    
    # getting the highest point candidate
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

the best candidate is:  0
pass 1 / 10 done.
the best candidate is:  0
pass 2 / 10 done.
the best candidate is:  0
pass 3 / 10 done.
the best candidate is:  0
pass 4 / 10 done.
the best candidate is:  0
pass 5 / 10 done.
the best candidate is:  0
pass 6 / 10 done.
the best candidate is:  0
pass 7 / 10 done.
the best candidate is:  1
pass 8 / 10 done.
the best candidate is:  0
pass 9 / 10 done.
the best candidate is:  1
pass 10 / 10 done.


In [5]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
 # convert pcd.PointCloud to numpy array
pcd_np = np.asarray(pcd.points)
print(pcd_np)

In [ ]:
# data exporting to arrays
pcd_np.shape
pcd_npx = pcd_np[:,0]
pcd_npy = pcd_np[:,1]
pcd_npz = pcd_np[:,2]

In [ ]:
# array plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(pcd_npx, pcd_npy, pcd_npz, marker='o')

In [ ]:

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# RANSAC implementation for planar shape detection in point clouds
# segment_plane()
#      *distance_threshold >> the plane to consider a point inlier or outlier
#      *ransac_n >> the number of sampled points drawn to estimate each plane candidate 
#      *num_iterations >> 
plane_model, inliers = pcd.segment_plane(distance_threshold=0.03, ransac_n=3, num_iterations=20000)
# select the points based on the indexes captured in >> inliers
# optionally select all the others as >> outliers
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
# paint the inliers in red
# outlier grey
inlier_cloud.paint_uniform_color([1, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
1+2

In [ ]:
# select the points based on the indexes captured in >> inliers
# optionally select all the others as >> outliers
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)


In [ ]:
# paint the inliers in red
# outlier grey
inlier_cloud.paint_uniform_color([1, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

# RANSAC on multiple Plane 

In [ ]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.15, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [ ]:
max_label

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
# Clustering with dbscan
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10))

In [ ]:
max_label = labels.max()
max_label

In [ ]:
# labels are valued in term of -1 to n
# -1 represents noise
# 0-n represetns each label
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label 
if max_label > 0 else 1))

# removing noise settting color to 0
colors[labels < 0] = 0

pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd])

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
# each plane is stored  in segments
# separating rest of the PCD in rest
# iterating rest again to find another plane
segment_models={}
segments={}

# initializing plane quantity for iteration
max_plane_idx=20

# initializing the pcd to rest
rest=pcd

for i in range(max_plane_idx):
    # taking color from tab20 for pcd plane to be colorized
    colors = plt.get_cmap("tab20")(i)
    
    # applying RANSAC to find plane ransac_n=3 distance_threshold=0.01 num_iteration>> iteration for RANSAC
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.03,ransac_n=3,num_iterations=20000)
    
    # storing plane to segment && applying color
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    
    # seperating others to rest for next iteration
    rest = rest.select_by_index(inliers, invert=True)
    
    
    print("pass",i,"/",max_plane_idx,"done.")

o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [ ]:
''' After this plane Segmentation there are some line artifact as because we fit all the points to RANSAC plane candidates 
    (which have no limit extent in the euclidean space) independently of the points density continuity.
    So after figuring out each plane we use DBSCAN to cluster the plane which is limited but plane segment and rest of 
    the things are same '''


Line Artifact Solution

In [ ]:
segment_models={}
segments={}
max_plane_idx=30
rest=pcd
d_threshold=0.1
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.05,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    # Clustering the plane Using DBSCAN
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    
    #  no of cluster found in that plane segment named as candidates
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    
    # getting the highest point candidate
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

In [ ]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [ ]:
# Rest of the planes are labeled , clustered  and colored 

labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5)) 
max_label = labels.max() 
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 
else 1)) 
colors[labels < 0] = 0 
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [ ]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

# o3d.visualization.draw_geometries([segments.values()])
# o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=0.3199,front=[0.30159062875123849, 0.94077325609922868, 0.15488309545553303],lookat=[-3.9559999108314514, -0.055000066757202148, -0.27599999308586121],up=[-0.044411423633999815, -0.138726419067636, 0.98753122516983349])
# o3d.visualization.draw_geometries([rest])